In [2]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split
import cv2

caltech_dir = "D:\\Drebin\\dex_exper\\dex_dataset_20"

categories =[]

with open("D:\\Drebin\\\\fam_20.txt","r") as f:
    lines = f.readlines()
    for line in lines:
        categories.append(line.split('\n')[0])
    
nb_classes = len(categories)
print(categories)   

image_w = 64
image_h = 64

pixels = image_h * image_w

X = []
y = []

for idx, cat in enumerate(categories):
    
    #one-hot 돌리기.
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

    image_dir = caltech_dir + "\\" + cat + "\\"
    files = glob.glob(image_dir+"*.jpg")
        
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("L")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        X.append(data)
        y.append(label)

['Adrd', 'BaseBridge', 'DroidDream', 'DroidKungFu', 'ExploitLinuxLotoor', 'FakeDoc', 'FakeInstaller', 'FakeRun', 'Gappusin', 'Geinimi', 'GinMaster', 'Glodream', 'Iconosys', 'Imlog', 'Kmin', 'MobileTx', 'Opfake', 'Plankton', 'SendPay', 'SMSreg']


In [8]:
X = np.array(X)
y = np.array(y)


X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y,test_size=0.2,random_state =42)
xy = (X_train, X_test, y_train, y_test)
np.save("D:\\Drebin\\dex_exper\\multi_code_data.npy20", xy)

print("ok", len(y))

ok 4639


In [9]:
X_train = X_train.reshape(X_train.shape[0],64,64, 1).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], 64, 64, 1).astype('float32') / 255

In [10]:
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D,Conv3D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K
from keras.optimizers import SGD, RMSprop

import tensorflow as tf
from keras import optimizers
from keras.utils import np_utils
from keras.applications.inception_v3 import InceptionV3
import sys

In [11]:
with K.tf_ops.device('/device:GPU:0'):
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3,3), input_shape = (64,64,1), activation='relu'))
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.5))
    model.add(Conv2D(150, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(20, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='Adamax', metrics=['accuracy'])
    
    model_dir = './model/'
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    
    modelName = model_dir + 'cnn_mnist.model'
    
    checkpointer = ModelCheckpoint(filepath=modelName, monitor='val_loss', verbose=1, save_best_only = True)
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=7)

In [12]:
history = model.fit(X_train, y_train, epochs=100, batch_size=10, validation_split = 0.15, callbacks=[early_stopping, checkpointer])

Train on 3154 samples, validate on 557 samples
Epoch 1/100
3154/3154 [==============================] - 3s 894us/step - loss: 2.3152 - accuracy: 0.2854 - val_loss: 2.0086 - val_accuracy: 0.3698

Epoch 00001: val_loss improved from inf to 2.00858, saving model to ./model/cnn_mnist.model
Epoch 2/100
3154/3154 [==============================] - 3s 826us/step - loss: 1.7412 - accuracy: 0.4876 - val_loss: 1.4863 - val_accuracy: 0.6050

Epoch 00002: val_loss improved from 2.00858 to 1.48625, saving model to ./model/cnn_mnist.model
Epoch 3/100
3154/3154 [==============================] - 3s 826us/step - loss: 1.3604 - accuracy: 0.6240 - val_loss: 1.2245 - val_accuracy: 0.6661

Epoch 00003: val_loss improved from 1.48625 to 1.22448, saving model to ./model/cnn_mnist.model
Epoch 4/100
3154/3154 [==============================] - 3s 820us/step - loss: 1.1564 - accuracy: 0.6782 - val_loss: 1.0777 - val_accuracy: 0.7038

Epoch 00004: val_loss improved from 1.22448 to 1.07773, saving model to ./mod

3154/3154 [==============================] - 3s 875us/step - loss: 0.1694 - accuracy: 0.9423 - val_loss: 0.4654 - val_accuracy: 0.9013

Epoch 00037: val_loss did not improve from 0.41369
Epoch 38/100
3154/3154 [==============================] - 3s 836us/step - loss: 0.1505 - accuracy: 0.9540 - val_loss: 0.4917 - val_accuracy: 0.9013

Epoch 00038: val_loss did not improve from 0.41369
Epoch 39/100
3154/3154 [==============================] - 3s 841us/step - loss: 0.1600 - accuracy: 0.9524 - val_loss: 0.4324 - val_accuracy: 0.9031

Epoch 00039: val_loss did not improve from 0.41369
Epoch 40/100
3154/3154 [==============================] - 3s 846us/step - loss: 0.1408 - accuracy: 0.9537 - val_loss: 0.4538 - val_accuracy: 0.9013

Epoch 00040: val_loss did not improve from 0.41369


In [13]:
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix

y_pred = model.predict(X_test)
y_test_class = np.argmax(y_test,axis=1)
y_pred_class = np.argmax(y_pred,axis=1)

print(classification_report(y_test_class,y_pred_class))
print(confusion_matrix(y_test_class,y_pred_class))

              precision    recall  f1-score   support

           0       0.83      0.56      0.67        18
           1       0.96      0.80      0.87        65
           2       0.92      0.75      0.83        16
           3       0.88      0.91      0.90       133
           4       0.75      0.46      0.57        13
           5       0.96      0.88      0.92        26
           6       0.98      0.98      0.98       185
           7       0.80      1.00      0.89        12
           8       0.89      0.67      0.76        12
           9       0.54      0.78      0.64        18
          10       0.69      0.81      0.74        68
          11       0.60      0.43      0.50        14
          12       0.94      0.97      0.95        31
          13       1.00      1.00      1.00         9
          14       0.91      1.00      0.95        29
          15       1.00      1.00      1.00        14
          16       0.97      0.97      0.97       120
          17       0.96    

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import RMSprop, Adadelta, Nadam, Adagrad, SGD, Adam
opt = RMSprop()
with K.tf_ops.device('/device:GPU:0'):
    model = InceptionV3(weights=None, include_top=True, classes=20)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    model_dir = './model/'
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)

    modelName = model_dir + 'cnn_mnist.model'
    checkpointer = ModelCheckpoint(filepath=modelName, monitor='val_loss', verbose=1, save_best_only = True)

    early_stopping = EarlyStopping(monitor='val_loss', patience=7)

In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=20,validation_split = 0.2, callbacks=[early_stopping, checkpointer])

In [7]:
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix

y_pred = model.predict(X_test)
y_test_class = np.argmax(y_test,axis=1)
y_pred_class = np.argmax(y_pred,axis=1)
print(classification_report(y_test_class,y_pred_class))

              precision    recall  f1-score   support

           0       0.70      0.39      0.50        18
           1       1.00      0.80      0.89        65
           2       0.86      0.75      0.80        16
           3       0.75      0.82      0.78       133
           4       0.10      0.08      0.09        13
           5       0.89      0.96      0.93        26
           6       0.96      0.74      0.84       185
           7       1.00      0.92      0.96        12
           8       0.53      0.75      0.62        12
           9       0.87      0.72      0.79        18
          10       0.83      0.56      0.67        68
          11       0.22      0.29      0.25        14
          12       0.36      0.94      0.52        31
          13       0.73      0.89      0.80         9
          14       0.94      1.00      0.97        29
          15       0.93      0.93      0.93        14
          16       0.91      0.96      0.93       120
          17       0.89    